##### Copyright 2024 Google LLC.

In [1]:
!pip install flask spacy

In [2]:
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 38.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [4]:
!pip install flask pyngrok

In [5]:
!ngrok authtoken 2fhIpIGUmKhPG1jHGdjHzCnyou0_2JXzxGdmKFQunq6eM8syr

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [19]:
import spacy
import json
from flask import Flask, request, jsonify
import re
from datetime import datetime, timedelta
import threading
import requests
import time

# Load SpaCy English model
nlp = spacy.load("en_core_web_sm")

app = Flask(__name__)

class StandardizedChoreParser:
    def __init__(self):
        # Define chore categories with associated keywords
        self.categories = {
            "cleaning": ["clean", "dust", "vacuum", "mop", "scrub", "sweep", "wipe", "wash", "tidy"],
            "bedroom": ["bed", "sheets", "pillow", "blanket", "bedroom"],
            "bathroom": ["toilet", "shower", "bath", "sink", "bathroom"],
            "kitchen": ["dishes", "kitchen", "countertop", "stove", "oven", "fridge", "garbage", "trash"],
            "laundry": ["clothes", "laundry", "fold", "iron", "washing"],
            "outside": ["yard", "lawn", "garden", "mow", "rake", "leaves", "snow", "shovel"],
            "pets": ["dog", "cat", "feed", "walk", "pet", "litter"],
            "homework": ["homework", "study", "reading", "practice", "school", "assignment"],
            "organization": ["organize", "sort", "arrange", "declutter", "put away"],
            "other": []  # Default category
        }

        # Common date-related words
        self.date_keywords = {
            "today": 0,
            "tonight": 0,
            "tomorrow": 1,
            "next day": 1,
            "day after tomorrow": 2,
            "weekend": [5, 6],  # Saturday and Sunday
        }

        # Time of day approximations
        self.time_of_day = {
            "morning": "09:00",
            "afternoon": "15:00",
            "evening": "19:00",
            "night": "21:00",
            "noon": "12:00",
            "midnight": "00:00",
            "dinner": "18:00",
            "breakfast": "08:00",
            "lunch": "13:00",
            "bedtime": "21:00"
        }

        # Days of the week
        self.days_of_week = {
            "monday": 0, "tuesday": 1, "wednesday": 2,
            "thursday": 3, "friday": 4, "saturday": 5, "sunday": 6
        }

    def extract_chore_description(self, doc):
        """Extract the main chore description from the text"""
        # Try to find verb + object pairs which likely describe the core task
        for token in doc:
            if token.pos_ == "VERB":
                # Find direct objects of the verb
                obj_text = ""
                for child in token.children:
                    if child.dep_ in ["dobj", "pobj"]:
                        # Include the object and any of its modifiers
                        modifiers = []
                        for grandchild in child.children:
                            if grandchild.dep_ in ["amod", "compound", "det"]:
                                modifiers.append(grandchild.text)

                        obj_phrase = " ".join(modifiers + [child.text])
                        obj_text = f"{token.lemma_} {obj_phrase}".strip()
                        break

                if obj_text:
                    return obj_text

        # Fallback: return the first 5-10 words as description
        return " ".join([token.text for token in doc[:min(10, len(doc))]])

    def determine_category(self, doc):
        """Determine the category of the chore"""
        text_lower = doc.text.lower()
        best_match = "other"
        max_matches = 0

        for category, keywords in self.categories.items():
            matches = sum(1 for keyword in keywords if keyword in text_lower)
            if matches > max_matches:
                max_matches = matches
                best_match = category

        return best_match

    def extract_child_assignee(self, doc):
        """Extract the child assigned to the task"""
        # Check for named entities that are persons
        for ent in doc.ents:
            if ent.label_ == "PERSON":
                return ent.text

        # Look for possessive pronouns that might indicate a child
        for token in doc:
            if token.text.lower() in ["your", "his", "her", "their"]:
                for child in token.children:
                    if child.pos_ == "NOUN":
                        return f"{token.text} {child.text}"

        # Look for pronouns that might be referring to a child
        pronouns = ["he", "she", "they", "him", "her", "them"]
        for token in doc:
            if token.text.lower() in pronouns:
                return token.text

        return None  # No specific assignee found

    def extract_due_date(self, doc):
        """Extract due date information and return a standardized date format YYYY-MM-DD"""
        text_lower = doc.text.lower()
        today = datetime.now()
        due_date = today  # Default to today if no date is specified
        time_str = "23:59"  # Default end of day
        has_date = False

        # Process specific dates from entities
        for ent in doc.ents:
            if ent.label_ == "DATE":
                try:
                    # Try parsing common date formats
                    date_text = ent.text.lower()

                    # Handle days of week in entities
                    for day_name, day_num in self.days_of_week.items():
                        if day_name in date_text:
                            today_weekday = today.weekday()
                            days_until_day = (day_num - today_weekday) % 7
                            # If the day is today and already passed, move to next week
                            if days_until_day == 0 and datetime.now().hour >= 12:
                                days_until_day = 7
                            due_date = today + timedelta(days=days_until_day)
                            has_date = True
                            break

                    # Handle "this weekend", "next week", etc.
                    if "weekend" in date_text:
                        today_weekday = today.weekday()
                        days_until_saturday = (5 - today_weekday) % 7
                        due_date = today + timedelta(days=days_until_saturday)
                        has_date = True
                    elif "next week" in date_text:
                        due_date = today + timedelta(days=7)
                        has_date = True
                    elif "month" in date_text:
                        due_date = today + timedelta(days=30)
                        has_date = True

                    # If we couldn't parse it, continue to other methods
                    if not has_date:
                        continue

                except (ValueError, TypeError):
                    # If parsing fails, continue to other methods
                    continue

        # Check for relative dates if no entity was found
        if not has_date:
            for keyword, day_offset in self.date_keywords.items():
                if keyword in text_lower:
                    if isinstance(day_offset, list):
                        # Handle weekend case
                        today_weekday = today.weekday()
                        days_until_saturday = (5 - today_weekday) % 7
                        due_date = today + timedelta(days=days_until_saturday)
                    else:
                        due_date = today + timedelta(days=day_offset)
                    has_date = True
                    break

        # Check for days of week if no relative date was found
        if not has_date:
            for day_name, day_num in self.days_of_week.items():
                if day_name in text_lower:
                    today_weekday = today.weekday()
                    days_until_day = (day_num - today_weekday) % 7
                    # If the day is today and already passed, move to next week
                    if days_until_day == 0 and datetime.now().hour >= 12:
                        days_until_day = 7
                    due_date = today + timedelta(days=days_until_day)
                    has_date = True
                    break

        # Check for specific time patterns (e.g., "5pm")
        time_pattern = r"\b(\d{1,2}):?(\d{2})?\s*(am|pm|AM|PM)?\b"
        time_match = re.search(time_pattern, doc.text)
        if time_match:
            hour = int(time_match.group(1))
            minute = time_match.group(2) or "00"
            period = (time_match.group(3) or "").lower()

            # Adjust hour for PM
            if period == "pm" and hour < 12:
                hour += 12
            elif period == "am" and hour == 12:
                hour = 0

            time_str = f"{hour:02d}:{minute}"
        else:
            # Check for time of day references
            for time_ref, time_val in self.time_of_day.items():
                if time_ref in text_lower:
                    time_str = time_val
                    break

        # Combine date and time
        return f"{due_date.strftime('%Y-%m-%d')}T{time_str}:00"

    def parse_chore(self, text):
        """Parse a chore task description into the required fields"""
        doc = nlp(text)

        chore_data = {
            "chore_description": self.extract_chore_description(doc),
            "category": self.determine_category(doc),
            "child_assignee": self.extract_child_assignee(doc),
            "due_date": self.extract_due_date(doc)
        }

        return chore_data

# Create an instance of the parser
parser = StandardizedChoreParser()

@app.route('/parse', methods=['POST'])
def parse_chore():
    """API endpoint to parse a chore task"""
    data = request.json

    if not data or 'chore' not in data:
        return jsonify({"error": "Missing chore text"}), 400

    chore_text = data['chore']
    parsed_data = parser.parse_chore(chore_text)

    return jsonify(parsed_data)

@app.route('/batch_parse', methods=['POST'])
def batch_parse_chores():
    """API endpoint to parse multiple chore tasks"""
    data = request.json

    if not data or 'chores' not in data:
        return jsonify({"error": "Missing chores list"}), 400

    chores = data['chores']
    parsed_chores = [parser.parse_chore(chore) for chore in chores]

    return jsonify({"chores": parsed_chores})

# Run Flask in a separate thread
def run_flask():
    app.run(host='0.0.0.0', port=5000)

thread = threading.Thread(target=run_flask)
thread.start()

# Wait for Flask to start
time.sleep(2)

# Simulate the curl command
payload = {"chore": "Tyler is responsible for taking out recycling on Thursday."}
headers = {"Content-Type": "application/json"}
response = requests.post('http://localhost:5000/parse', json=payload, headers=headers)

# Print the response
print("Status Code:", response.status_code)
print("Response JSON:", response.json())

 * Serving Flask app '__main__'
 * Debug mode: off


Address already in use
Port 5000 is in use by another program. Either identify and stop that program, or start the server with a different port.
INFO:werkzeug:127.0.0.1 - - [28/Feb/2025 17:26:05] "POST /parse HTTP/1.1" 200 -


Status Code: 200
Response JSON: {'category': 'other', 'child_assignee': 'Tyler', 'chore_description': 'take recycling', 'due_date': '2025-03-06T23:59:00'}


In [ ]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Gemini API: Prompting Quickstart

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/google-gemini/cookbook/blob/main/quickstarts/Prompting.ipynb"><img src="https://github.com/google-gemini/cookbook/blob/main/images/colab_logo_32px.png?raw=1" />Run in Google Colab</a>
  </td>
</table>

This notebook contains examples of how to write and run your first prompts with the Gemini API.

In [ ]:
!pip install -U -q "google-generativeai>=0.7.2" # Install the Python SDK

In [ ]:
import google.generativeai as genai

## Set up your API key

To run the following cell, your API key must be stored it in a Colab Secret named `GOOGLE_API_KEY`. If you don't already have an API key, or you're not sure how to create a Colab Secret, see the [Authentication](https://github.com/google-gemini/cookbook/blob/main/quickstarts/Authentication.ipynb) quickstart for an example.

In [ ]:
from google.colab import userdata
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

## Run your first prompt

Use the `generate_content` method to generate responses to your prompts. You can pass text directly to generate_content, and use the `.text` property to get the text content of the response.

In [ ]:
model = genai.GenerativeModel('gemini-2.0-flash')
response = model.generate_content("Give me python code to sort a list")
print(response.text)

```python
# There are several ways to sort a list in Python. Here are the most common:

# 1. Using the `sort()` method (in-place sorting):
def sort_list_in_place(my_list):
  """Sorts a list in place using the sort() method.  Modifies the original list."""
  my_list.sort()  # Sorts the list in ascending order by default.
  # To sort in descending order:  my_list.sort(reverse=True)
  return my_list  # Returns the *same* list, now sorted.  Not strictly necessary, but can be convenient.


# 2. Using the `sorted()` function (creates a new sorted list):
def sort_list_new(my_list):
  """Creates a new sorted list from the original list using the sorted() function.
     The original list remains unchanged."""
  new_list = sorted(my_list) # Returns a new sorted list, leaving the original untouched.
  # To sort in descending order:  new_list = sorted(my_list, reverse=True)
  return new_list


# 3. Sorting with a custom key (using `sort()` or `sorted()`):

def sort_list_by_length(my_list):
  """So

## Use images in your prompt

Here you will download an image from a URL and pass that image in our prompt.

First, you download the image and load it with PIL:

In [ ]:
!curl -o image.jpg "https://storage.googleapis.com/generativeai-downloads/images/jetpack.jpg"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  349k  100  349k    0     0  2364k      0 --:--:-- --:--:-- --:--:-- 2375k


In [ ]:
import PIL.Image
img = PIL.Image.open('image.jpg')
img

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
prompt = """This image contains a sketch of a potential product along with some notes.
Given the product sketch, describe the product as thoroughly as possible based on what you
see in the image, making sure to note all of the product features. Return output in json format:
{description: description, features: [feature1, feature2, feature3, etc]}"""

Then you can include the image in our prompt by just passing a list of items to `generate_content`.

In [ ]:
model = genai.GenerativeModel('gemini-2.0-flash')
response = model.generate_content([prompt, img])
print(response.text)

```json
{
  "description": "The product is a jetpack backpack designed to look like a normal, lightweight backpack. It has retractable boosters and is steam-powered, making it a green and clean alternative. It is designed to fit an 18\" laptop.  The backpack is USB-C chargeable and has a 15-minute battery life. It also features padded strap support.",
  "features": [
    "Lightweight",
    "Looks like a normal backpack",
    "Fits 18\" laptop",
    "Retractable Boosters",
    "Steam-powered (Green/Clean)",
    "USB-C Charging",
    "15-minute battery life",
    "Padded Strap Support"
  ]
}
```


## Have a chat

The Gemini API enables you to have freeform conversations across multiple turns.

The [ChatSession](https://ai.google.dev/api/python/google/generativeai/ChatSession) class will store the conversation history for multi-turn interactions.

In [ ]:
model = genai.GenerativeModel('gemini-2.0-flash')
chat = model.start_chat(history=[])

In [ ]:
response = chat.send_message("In one sentence, explain how a computer works to a young child.")
print(response.text)

A computer is like a super-smart toy that follows your instructions to play games, draw pictures, or tell you stories!



You can see the chat history:

In [ ]:
print(chat.history)

[parts {
  text: "In one sentence, explain how a computer works to a young child."
}
role: "user"
, parts {
  text: "A computer is like a super-smart toy that follows your instructions to play games, draw pictures, or tell you stories!\n"
}
role: "model"
]


You can keep sending messages to continue the conversation:

In [ ]:
response = chat.send_message("Okay, how about a more detailed explanation to a high schooler?")
print(response.text)

A computer operates by executing instructions stored in its memory, processing data through the CPU based on algorithms and logic, and utilizing input/output devices to interact with the user and external environment, all coordinated by an operating system that manages resources and provides a platform for applications.



## Set the temperature

Every prompt you send to the model includes parameters that control how the model generates responses. Use a `genai.GenerationConfig` to set these, or omit it to use the defaults.

Temperature controls the degree of randomness in token selection. Use higher values for more creative responses, and lower values for more deterministic responses.

You can set the `generation_config` when creating the model.

In [ ]:
model = genai.GenerativeModel(
    'gemini-2.0-flash',
    generation_config=genai.GenerationConfig(
        max_output_tokens=2000,
        temperature=0.9,
    ))

Or, set the `generation_config` on an individual call to `generate_content`. Any values set there override values on the model constructor.

Note: Although you can set the `candidate_count` in the generation_config, gemini-2.0-flash models will only return a single candidate at the this time.

In [ ]:
response = model.generate_content(
    'Give me a numbered list of cat facts.',
    # Limit to 5 facts.
    generation_config = genai.GenerationConfig(stop_sequences=['\n6'])
)

In [ ]:
print(response.text)

Okay, here's a numbered list of fascinating cat facts:

1.  **Cats have a third eyelid:** This is called a nictitating membrane, and it's a translucent eyelid that protects and moistens the eye. It's usually only visible when a cat is sleepy or sick.

2.  **A cat's nose print is unique:** Just like human fingerprints, no two cats have the same nose print pattern.

3.  **Cats can't taste sweetness:** They lack the taste receptor gene that allows mammals to taste sweet flavors.

4.  **Cats are crepuscular animals:** This means they are most active during dawn and dusk, which is when their prey (rodents) are also most active.

5.  **Cats have flexible bodies thanks to their "floating" clavicle:** Unlike humans, a cat's collarbone isn't attached to other bones, allowing for greater flexibility and range of motion.



## Learn more

There's lots more to learn!

* For more fun prompts, check out [Market a Jetpack](https://github.com/google-gemini/cookbook/blob/main/examples/Market_a_Jet_Backpack.ipynb).
* Check out the [safety quickstart](https://github.com/google-gemini/cookbook/blob/main/quickstarts/Safety.ipynb) next to learn about the Gemini API's configurable safety settings, and what to do if your prompt is blocked.
* For lots more details on using the Python SDK, check out this [detailed quickstart](https://ai.google.dev/tutorials/python_quickstart).